In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import networkx as nx

kaggle_data_path=os.path.join(os.sep, "kaggle", "input", "CORD-19-research-challenge")

In [2]:
df = pd.read_csv(kaggle_data_path+"/metadata.csv")

In [3]:
datafiles = []
for dirname, _, filenames in os.walk(kaggle_data_path):
    for filename in filenames:
        ifile = os.path.join(dirname, filename)
        if ifile.split(".")[-1] == "json":
            datafiles.append(ifile)

In [4]:
citations = []
filter_words = ['proc.', 'magazine']#,'magazine']#'copyright','nan']
remove_names=["publisher's note", "world health organization", "fields virology", "united states census", "geneva: world health organization"]
for file in datafiles:
    '''id and title of a single document'''
    with open(file,'r')as f:
        doc = json.load(f)
    reftitle = doc['metadata']['title'].lower()
    
    '''collect bib-entries of a single document'''
    for key,value in doc['bib_entries'].items():
#         if any(x not in value['title'].lower() for x in filter_words):
        value['title'] = value['title'].lower().split('proc.')[0]
        if value['title'] in remove_names: 
            continue
        if (len(set(value['title'].lower().split(' ')).intersection(set(filter_words))) == 0) and len(value['title'].lower()) > 0:
            citations.append({"title": reftitle, "citation": value['title'].lower()})
        else:
            if len(value['title']) > 0: print(value['title'])

framing women's health with a sense-making approach: magazine coverage of breast cancer and implants
framing women’s health with a sense-making approach: magazine coverage of breast cancer and implants
on the spongiae ciliatae as infusoria flagellata: or observations on the 717 structure, animality and relationship of leucosolenia botryoides bowerbank. the annals and 718 magazine of natural history
would everyone wearing face masks help us slow the pandemic? : science 314 magazine
genotypic anomaly in ebola virus strains circulating in magazine wharf area
issues continue to dog the testing of ebola drugs and vaccines. science magazine
foot and mouth disease -how does switzerland react? federal veterinary office magazine
is it worth imagining airborne ebola? national geographic magazine
kml gets two thumbs up from file format experts. directions magazine
severe acute respiratory syndrome (sars). the voice magazine
genotypic anomaly in ebola virus strains circulating in magazine wharf ar

In [5]:
dfc = pd.DataFrame(citations)

In [6]:
len(dfc)

3083528

In [7]:
dfc.to_csv('title_citation.zip', compression='zip', index=False)

In [8]:
!ls -rtl --block-size=MB

total 104MB
---------- 1 root root   1MB Apr 14 15:19 __notebook__.ipynb
-rw-r--r-- 1 root root 104MB Apr 14 15:24 title_citation.zip


In [9]:
G = nx.from_pandas_edgelist(dfc,source='title',target='citation',create_using=nx.DiGraph)

In [10]:
%time pr = nx.pagerank(G)

CPU times: user 1min 2s, sys: 1.39 s, total: 1min 3s
Wall time: 1min 3s


In [11]:
pagerank = pd.DataFrame(pr.items(), columns=["title", "pagerank"]).sort_values(by="pagerank", ascending=False)

In [12]:
pagerank.head(50)

,title,pagerank
297,isolation of a novel coronavirus from a man wi...,0.000023
448,identification of a novel coronavirus in patie...,0.000023
565,a novel coronavirus associated with severe acu...,0.000022
5587,coronavirus as a possible cause of severe acut...,0.000018
1244,"early transmission dynamics in wuhan, china, o...",0.000016
5407,a novel coronavirus from patients with pneumon...,0.000015
5594,characterization of a novel coronavirus associ...,0.000015
8018,a major outbreak of severe acute respiratory s...,0.000014
8021,clinical features of patients infected with 20...,0.000012
1863,epidemiological and clinical characteristics o...,0.000011


In [13]:
pagerank.iloc[0]

title       isolation of a novel coronavirus from a man wi...
pagerank                                           2.2989e-05
Name: 297, dtype: object

In [14]:
pagerank.iloc[6000]

title       priorities for pe development: -review pe trai...
pagerank                                          1.00104e-06
Name: 221662, dtype: object

In [15]:
pagerank.iloc[1]["title"]

'identification of a novel coronavirus in patients with severe acute respiratory syndrome'

In [16]:
len(pagerank)

1398451

In [17]:
title = pagerank.iloc[0]["title"]
print(f"The paper <<{title}>> \n is cited {G.in_degree[title]} times. Its annexe contain {G.out_degree[title]} references.")

The paper <<isolation of a novel coronavirus from a man with pneumonia in saudi arabia>> 
 is cited 1151 times. Its annexe contain 0 references.
